In [1]:
conda activate ti_qec


Note: you may need to restart the kernel to use updated packages.


In [2]:
%run qsu.ipynb  # color-printing functions
import numpy as np
from qecsim import paulitools as pt
from qecsim.models.generic import DepolarizingErrorModel, BitFlipErrorModel, PhaseFlipErrorModel
from qecsim.models.planar import PlanarCode, PlanarMWPMDecoder, PlanarMPSDecoder
from ti_decoder import ti_calc
from gen_hard_synd import gen_hard_synd

# initialise models
d = 17
my_code = PlanarCode(d, d)
my_error_model = PhaseFlipErrorModel()
my_decoder = PlanarMWPMDecoder()
# print models
print(my_code)
print(my_error_model)
print(my_decoder)

PlanarCode(17, 17)
PhaseFlipErrorModel()
PlanarMWPMDecoder()


In [4]:
# set physical error probability to 10%
error_probability = 0.1

synd_lst, err_lst = gen_hard_synd(d=d, error_probability=error_probability, num_samples=1, seed=1)

error = err_lst[0]
qsu.print_pauli('error:\n{}'.format(my_code.new_pauli(error)))

In [5]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

Please add the following code to PlanarMPSDecoder class
```python
def calc_coset_probabilities(self, code, syndrome,
            error_model=DepolarizingErrorModel(),  # noqa: B008
            error_probability=0.1, **kwargs):
    """
    See :meth:`qecsim.model.Decoder.decode`

    Note: The optional keyword parameters ``error_model`` and ``error_probability`` are used to determine the prior
    probability distribution for use in the decoding algorithm. Any provided error model must implement
    :meth:`~qecsim.model.ErrorModel.probability_distribution`.

    :param code: Planar code.
    :type code: PlanarCode
    :param syndrome: Syndrome as binary vector.
    :type syndrome: numpy.array (1d)
    :param error_model: Error model. (default=DepolarizingErrorModel())
    :type error_model: ErrorModel
    :param error_probability: Overall probability of an error on a single qubit. (default=0.1)
    :type error_probability: float
    :return: Recovery operation as binary symplectic vector.
    :rtype: numpy.array (1d)
    """
    # any recovery
    any_recovery = self.sample_recovery(code, syndrome)
    # probability distribution
    prob_dist = error_model.probability_distribution(error_probability)
    # coset probabilities, recovery operations
    coset_ps, recoveries = self._coset_probabilities(prob_dist, any_recovery)

    return coset_ps, recoveries
```


In [6]:
# from mps decoder
my_mps_decoder = PlanarMPSDecoder(chi=6)
coset_ps, recoveries = my_mps_decoder.calc_coset_probabilities(
    my_code,
    syndrome,
    error_model=my_error_model,
    error_probability=error_probability)

print('total prob of each coset', coset_ps)
print('Only the first and last are non-zero because only Z errors.')
print('The log of their ratio is:',
      np.log(float(coset_ps[3]) / float(coset_ps[0])))


total prob of each coset (mpf('2.671806286246112e-72'), mpf('0.0'), mpf('0.0'), mpf('2.8596372036597147e-72'))
Only the first and last are non-zero because only Z errors.
The log of their ratio is: 0.06794000953421651


In [7]:
recovery = recoveries[0].to_bsf()
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

In [8]:
ti_calc(recovery,
           error_probability=error_probability,
           d=d,
           num_int_steps=201)


l = 0.00, energy derivative = -19.0560
l = 0.01, energy derivative = -19.7320
l = 0.01, energy derivative = -19.0640
l = 0.01, energy derivative = -18.9640
l = 0.02, energy derivative = -18.1960
l = 0.03, energy derivative = -18.3360
l = 0.03, energy derivative = -18.8280
l = 0.04, energy derivative = -18.6240
l = 0.04, energy derivative = -18.7200
l = 0.04, energy derivative = -18.6840
l = 0.05, energy derivative = -19.7120
l = 0.06, energy derivative = -19.1120
l = 0.06, energy derivative = -18.1560
l = 0.07, energy derivative = -17.8240
l = 0.07, energy derivative = -17.4520
l = 0.07, energy derivative = -17.8560
l = 0.08, energy derivative = -17.3320
l = 0.09, energy derivative = -18.6000
l = 0.09, energy derivative = -17.5880
l = 0.10, energy derivative = -18.8560
l = 0.10, energy derivative = -18.1560
l = 0.10, energy derivative = -16.0000
l = 0.11, energy derivative = -15.6640
l = 0.12, energy derivative = -18.9000
l = 0.12, energy derivative = -16.6440
l = 0.12, energy derivati

-0.9547542019131596

In [9]:
ti_calc(recovery, error_probability=error_probability, d=d, num_int_steps=1001)

l = 0.00, energy derivative = -19.5880
l = 0.00, energy derivative = -19.6560
l = 0.00, energy derivative = -19.4120
l = 0.00, energy derivative = -18.5640
l = 0.00, energy derivative = -18.6960
l = 0.01, energy derivative = -19.3240
l = 0.01, energy derivative = -18.4480
l = 0.01, energy derivative = -18.9800
l = 0.01, energy derivative = -20.8280
l = 0.01, energy derivative = -20.2520
l = 0.01, energy derivative = -19.3920
l = 0.01, energy derivative = -19.5680
l = 0.01, energy derivative = -19.2560
l = 0.01, energy derivative = -19.2400
l = 0.01, energy derivative = -19.1680
l = 0.01, energy derivative = -18.5520
l = 0.02, energy derivative = -18.7840
l = 0.02, energy derivative = -19.3360
l = 0.02, energy derivative = -18.9680
l = 0.02, energy derivative = -18.7880
l = 0.02, energy derivative = -18.9920
l = 0.02, energy derivative = -18.6880
l = 0.02, energy derivative = -18.1320
l = 0.02, energy derivative = -20.1240
l = 0.02, energy derivative = -19.8560
l = 0.03, energy derivati

-0.3305451095423603

## Try starting configuration from MWPM

In [10]:
mwpm = PlanarMWPMDecoder()
mwpm_recovery = mwpm.decode(my_code, syndrome)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(mwpm_recovery)))

In [11]:
ti_calc(mwpm_recovery,
           error_probability=error_probability,
           d=d,
           num_int_steps=201)

l = 0.00, energy derivative = -19.0520
l = 0.01, energy derivative = -20.0120
l = 0.01, energy derivative = -19.2120
l = 0.01, energy derivative = -18.9280
l = 0.02, energy derivative = -18.5960
l = 0.03, energy derivative = -18.6680
l = 0.03, energy derivative = -19.5320
l = 0.04, energy derivative = -18.9840
l = 0.04, energy derivative = -18.5280
l = 0.04, energy derivative = -19.3240
l = 0.05, energy derivative = -19.1360
l = 0.06, energy derivative = -19.9240
l = 0.06, energy derivative = -18.8440
l = 0.07, energy derivative = -18.4560
l = 0.07, energy derivative = -18.4240
l = 0.07, energy derivative = -18.3080
l = 0.08, energy derivative = -18.8320
l = 0.09, energy derivative = -18.6200
l = 0.09, energy derivative = -20.1640
l = 0.10, energy derivative = -18.7640
l = 0.10, energy derivative = -18.6440
l = 0.10, energy derivative = -18.7280
l = 0.11, energy derivative = -17.0760
l = 0.12, energy derivative = -16.4640
l = 0.12, energy derivative = -18.8800
l = 0.12, energy derivati

-0.28808784333889775